In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt

output=[]

def uncode_table(table_path):
        # Load the table from the file
    with open(table_path, "r") as file:
        table_data = file.read()
    
    # Parse the table and create the dictionary
    data_dict = {}
    
    # Split the table into rows using the newline character
    rows = table_data.split("\n")
    
    dataset_name = ['Aquatic', 'Arial', 'Game', 'Medical', 'Surgical', 'Overall']
    
    # Iterate over each row
    rows = [r for r in rows if len(r)>10]
    delimiters = [" ", "&", "{", "}", "\\"]
    k=0
    for i in range(0, len(rows), 7):
        key = rows[i].split(' ')[1].replace('ourMethod', 'FOMO')
        values = rows[i+1:i+7]
        tmp={}
        for ii, row in enumerate(values):
            result = re.split("|".join(map(re.escape, delimiters)), row)
            tmp[dataset_name[ii]] = [float(r) for r in result if len(r)>1 and len(r)<5]
    
        data_dict[key]=tmp
        
    df = pd.DataFrame(data_dict)

    # Reset index to make index names a separate column
    df.reset_index(inplace=True)
    # Unpivot the table using pandas' melt() function
    unpivoted_df = pd.melt(df, id_vars=['index'], var_name='Category', value_name='Values')
    
    # Rename the columns to include 'Index' instead of 'index'
    unpivoted_df.rename(columns={'index': 'Index'}, inplace=True)
    unpivoted_df["Shots"]=num_shots
    return unpivoted_df
    
    
for num_shots in [1, 10, 100]:
    table_path = f"table-{num_shots}.txt"  # Replace with the actual path to your table file
    unpivoted_df = uncode_table(table_path)
    output.append(unpivoted_df)

df = pd.concat(output).reset_index()
df[['U', 'K', 'PK', 'CK']] = pd.DataFrame(df['Values'].apply(lambda x: pd.Series(x)).values)
df.drop('Values', axis=1, inplace=True)
df.rename(columns={'Category': 'Model', 'Index': 'Dataset'}, inplace=True)
df.to_csv("main_table.csv")


In [ ]:
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import seaborn as sns

# Set Seaborn style
sns.set_style('whitegrid')

metrics = ['U', 'K', 'PK', 'CK']
line_styles = ['-', '--', ':', '-.']
colors = ['blue', 'black', 'orange', 'red']
block_width = 10  # Adjust the width of the color blocks

# Create subplots for U and another subplot for K, PK, CK
fig, axs = plt.subplots(1, 2, figsize=(5, 3.5), sharex=True)
fig.subplots_adjust(hspace=0.4)  # Adjust the vertical spacing between subplots

# Plot for U
ax_u = axs[0]
grouped_data_u = df.groupby(['Model', 'Shots'])['U'].agg(['mean', 'std']).reset_index()
handles_u = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    subset = grouped_data_u[grouped_data_u['Model'] == model]
    shots = subset['Shots']
    mean = subset['mean']
    std = subset['std']
    line = ax_u.plot(shots, mean, marker='o', markersize=4, linestyle=line_styles[0], color=colors[i], label=model)
    handles_u.append(line[0])

#ax_u.set_ylabel('U-mAP')

# Plot for K, PK, CK
ax_kpkck = axs[1]
handles_kpkck = []
for j, metric in enumerate(['K', 'PK', 'CK']):
    grouped_data = df.groupby(['Model', 'Shots'])[metric].agg(['mean', 'std']).reset_index()
    for k, model in enumerate(grouped_data['Model'].unique()):
        subset = grouped_data[grouped_data['Model'] == model]
        shots = subset['Shots']
        mean = subset['mean']
        std = subset['std']
        line = ax_kpkck.plot(shots, mean, marker='o', markersize=4, linestyle=line_styles[j+1], color=colors[k])
        handles_kpkck.append(line[0])

#ax_kpkck.set_ylabel('K-mAP')

# Create custom legend handles for line styles
line_handles = []
for style in line_styles:
    line = mlines.Line2D([], [], color='black', linestyle=style)
    line_handles.append(line)

# Create color block handles for models
model_handles = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    color = colors[i]
    patch = mpatches.Rectangle((0, 0), 1, 0.6, facecolor=color, linewidth=1, edgecolor='k')  # Adjust the width and height
    model_handles.append(patch)

# Combine the U, K, PK, CK legend handles
all_handles = model_handles + line_handles
labels = [model for model in grouped_data_u['Model'].unique()] + metrics

# Create legend
fig.legend(handles=all_handles, labels=labels, bbox_to_anchor=(0.51, -0.07), loc='lower center', ncol=8, labelspacing=0.7, handlelength=1.6, handletextpad=0.3, columnspacing=0.9)
# Set the x-axis label for the last subplot
ax_kpkck.set_xlabel('Number of Shots')
ax_u.set_xlabel('Number of Shots')

# Adjust the spacing between subplots
fig.tight_layout()
# Adjust figure size
fig.set_size_inches(7.3, 3.5)

# Adjust layout
plt.tight_layout()

# Save figure
plt.savefig('my_figure.png', bbox_inches='tight')
# Show the plot
plt.show()


In [ ]:
uncode_table(table_path)

In [ ]:
 df['Dataset'].unique()

In [ ]:
# Define the metrics and datasets for plotting
metrics = ['U', 'K', 'PK', 'CK']
datasets = df['Dataset'].unique()

# Create subplots for each dataset/metric combination
fig, axs = plt.subplots(len(datasets), len(metrics), figsize=(12, 18))
fig.subplots_adjust(hspace=0.4, wspace=0.3)  # Adjust the vertical and horizontal spacing between subplots

# Iterate over each dataset and metric combination
for i, dataset in enumerate(datasets):
    for j, metric in enumerate(metrics):
        ax = axs[i, j] if len(datasets) > 1 else axs[j]  # Select the appropriate subplot
        
        # Filter the DataFrame for the current dataset and metric
        subset = df[(df['Dataset'] == dataset)]
        
        # Plot the metric against the number of shots
        for model, group in subset.groupby('Model'):
            ax.plot(group['Shots'], group[metric], marker='o', label=model)
        
        # Set the title and labels for the subplot
        ax.set_title(f'{dataset} - {metric}')
        ax.set_xlabel('Number of Shots')
        ax.set_ylabel(f'{metric} Values')
        
        # Add a legend to the subplot
        ax.legend()

# Adjust the spacing between subplots
plt.tight_layout()
plt.savefig("tmp.jpg")
# Show the plot
plt.show()

In [ ]:
# Define the metrics and datasets for plotting
metrics = ['U', 'K', 'PK', 'CK']
models = df.Model.unique()
# Create subplots for each dataset/metric combination
fig, axs = plt.subplots(1, len(metrics), figsize=(12, 4))
fig.subplots_adjust(hspace=0.4, wspace=0.3)  # Adjust the vertical and horizontal spacing between subplots

# Iterate over each dataset and metric combination
for j, metric in enumerate(metrics):
    ax = axs[j]  # Select the appropriate subplot
    
    # Filter the DataFrame for the current dataset and metric
    for model in models:
        subset = df[df.Model==model]
        
    
    # Plot the metric against the number of shots
    for model, group in subset.groupby('Model'):
        ax.plot(group['Shots'], group[metric], marker='o', label=model)
    
    # Set the title and labels for the subplot
    ax.set_title(f'{dataset} - {metric}')
    ax.set_xlabel('Number of Shots')
    ax.set_ylabel(f'{metric} Values')
    
    # Add a legend to the subplot
    ax.legend()

# Adjust the spacing between subplots
plt.tight_layout()
plt.savefig("tmp.jpg")
# Show the plot
plt.show()

In [ ]:
metrics = ['U', 'K', 'PK', 'CK']

# Create subplots for each metric
fig, axs = plt.subplots(1, len(metrics), figsize=(12, 3), sharex=True)
fig.subplots_adjust(hspace=0.9)  # Adjust the vertical spacing between subplots

# Iterate over each metric and plot the mean with error bars
for i, metric in enumerate(metrics):
    ax = axs[i]
    
    # Group by Model and compute mean and standard deviation for each Shots value
    grouped_data = df.groupby(['Model', 'Shots'])[metric].agg(['mean', 'std']).reset_index()
    
    # Iterate over each Model and plot the mean with error bars
    for model in grouped_data['Model'].unique():
        subset = grouped_data[grouped_data['Model'] == model]
        shots = subset['Shots']
        mean = subset['mean']
        std = subset['std']
        
        ax.plot(shots, mean, marker='o', linestyle='-', label=model)
    
    # Set the title and labels for the subplot
    ax.set_title(metric)
    ax.set_ylabel(f'{metric} Values')
    
    # Add a legend to the subplot
    ax.legend()

# Set the x-axis label for the last subplot
axs[-1].set_xlabel('Number of Shots')

# Show the plot
plt.show()

In [ ]:
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

metrics = ['U', 'K', 'PK', 'CK']
line_styles = ['-', '--', '-.', ':']
colors = ['blue', 'green', 'orange', 'red']
block_width = 10  # Adjust the width of the color blocks

# Create subplots for U and another subplot for K, PK, CK
fig, axs = plt.subplots(1, 2, figsize=(8, 4), sharex=True)
fig.subplots_adjust(hspace=0.4)  # Adjust the vertical spacing between subplots

# Plot for U
ax_u = axs[0]
grouped_data_u = df.groupby(['Model', 'Shots'])['U'].agg(['mean', 'std']).reset_index()
handles_u = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    subset = grouped_data_u[grouped_data_u['Model'] == model]
    shots = subset['Shots']
    mean = subset['mean']
    std = subset['std']
    line = ax_u.plot(shots, mean, marker='o', linestyle=line_styles[0], color=colors[i], label=model)
    handles_u.append(line[0])

ax_u.set_ylabel('U-mAP')

# Plot for K, PK, CK
ax_kpkck = axs[1]
handles_kpkck = []
for j, metric in enumerate(['K', 'PK', 'CK']):
    grouped_data = df.groupby(['Model', 'Shots'])[metric].agg(['mean', 'std']).reset_index()
    for k, model in enumerate(grouped_data['Model'].unique()):
        subset = grouped_data[grouped_data['Model'] == model]
        shots = subset['Shots']
        mean = subset['mean']
        std = subset['std']
        line = ax_kpkck.plot(shots, mean, marker='o', linestyle=line_styles[j+1], color=colors[k])
        handles_kpkck.append(line[0])

ax_kpkck.set_ylabel('K-mAP')

# Create custom legend handles for line styles
line_handles = []
for style in line_styles:
    line = mlines.Line2D([], [], color='black', linestyle=style)
    line_handles.append(line)

# Create color block handles for models
model_handles = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    color = colors[i]
    patch = mpatches.Rectangle((0, 0), 1, 0.6, facecolor=color, linewidth=1, edgecolor='k')  # Adjust the width and height
    model_handles.append(patch)

# Combine the U, K, PK, CK legend handles
all_handles = model_handles + line_handles
labels = [model for model in grouped_data_u['Model'].unique()] + metrics

# Create legend
fig.legend(handles=all_handles, labels=labels, bbox_to_anchor=(0.5, -0.10), loc='lower center', ncol=8, title='Models', labelspacing=0.4, handlelength=2, handletextpad=0.5)



# Set the x-axis label for the last subplot
ax_kpkck.set_xlabel('Number of Shots')

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

metrics = ['U', 'K', 'PK', 'CK']
line_styles = ['-', '--', '-.', ':']
colors = ['blue', 'green', 'orange', 'red']
block_width = 10  # Adjust the width of the color blocks

# Create subplots for U and another subplot for K, PK, CK
fig, axs = plt.subplots(1, 2, figsize=(8, 4), sharex=True)
fig.subplots_adjust(hspace=0.4)  # Adjust the vertical spacing between subplots

# Plot for U
ax_u = axs[0]
grouped_data_u = df.groupby(['Model', 'Shots'])['U'].agg(['mean', 'std']).reset_index()
handles_u = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    subset = grouped_data_u[grouped_data_u['Model'] == model]
    shots = subset['Shots']
    mean = subset['mean']
    std = subset['std']
    line = ax_u.plot(shots, mean, marker='o', linestyle=line_styles[0], color=colors[i], label=model)
    handles_u.append(line[0])

ax_u.set_ylabel('U-mAP')

# Plot for K, PK, CK
ax_kpkck = axs[1]
handles_kpkck = []
for j, metric in enumerate(['K', 'PK', 'CK']):
    grouped_data = df.groupby(['Model', 'Shots'])[metric].agg(['mean', 'std']).reset_index()
    for k, model in enumerate(grouped_data['Model'].unique()):
        subset = grouped_data[grouped_data['Model'] == model]
        shots = subset['Shots']
        mean = subset['mean']
        std = subset['std']
        line = ax_kpkck.plot(shots, mean, marker='o', linestyle=line_styles[j+1], color=colors[k])
        handles_kpkck.append(line[0])

ax_kpkck.set_ylabel('K-mAP')

# Create custom legend handles for line styles
line_handles = []
for style in line_styles:
    line = mlines.Line2D([], [], color='black', linestyle=style)
    line_handles.append(line)

# Create color block handles for models
model_handles = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    color = colors[i]
    patch = mpatches.Patch(color=color, linewidth=0, label=model)
    model_handles.append(patch)

# Combine the U, K, PK, CK legend handles
all_handles = model_handles + line_handles
labels = [handle.get_label() for handle in model_handles] + metrics + line_styles

# Create legend
fig.legend(handles=all_handles, labels=labels, bbox_to_anchor=(0.5, -0.10), loc='lower center', ncol=8, title='Models', labelspacing=0.4)



# Set the x-axis label for the last subplot
ax_kpkck.set_xlabel('Number of Shots')

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

# Set the style of the plots
sns.set(style="whitegrid")

metrics = ['U', 'K', 'PK', 'CK']
line_styles = ['-', '--', '-.', ':']
colors = ['blue', 'green', 'orange', 'red']

# Create a subplot for each metric
fig, axs = plt.subplots(1, 2, figsize=(8, 4), sharex=True)
fig.subplots_adjust(hspace=0.4)  # Adjust the vertical spacing between subplots

# Define a color palette
palette = dict(zip(df['Model'].unique(), colors))

# For each metric, create a separate subplot
for i, metric in enumerate(metrics):
    ax = axs[0] if metric == 'U' else axs[1]
    
    # Create a new dataframe where each row is a unique combination of Model, Shots, and the metric
    data = df.melt(id_vars=['Model', 'Shots'], value_vars=[metric], var_name='Metric', value_name='Value')
    
    # Use Seaborn to create the plot
    sns.lineplot(data=data, x='Shots', y='Value', hue='Model', style='Metric', markers=True, dashes=False, ax=ax, legend=False, errorbar=None, palette=palette)

    # Set the y-label of the plot
    ax.set_ylabel(f'{metric}-mAP')

# Set the x-axis label for the last subplot
axs[1].set_xlabel('Number of Shots')

# Create custom legend handles for line styles
line_handles = []
for i, style in enumerate(line_styles[:len(metrics)]):
    line = mlines.Line2D([], [], color='black', linestyle=style, label=metrics[i])
    line_handles.append(line)

# Create color block handles for models
model_handles = []
for i, model in enumerate(df['Model'].unique()):
    color = colors[i]
    patch = mpatches.Rectangle((0, 0), 1, 0.6, facecolor=color, linewidth=1, edgecolor='k')  # Adjust the width and height
    model_handles.append(patch)

# Combine the U, K, PK, CK legend handles
all_handles = model_handles + line_handles
labels = [model for model in df['Model'].unique()] + metrics

# Create legend
fig.legend(handles=all_handles, labels=labels, bbox_to_anchor=(0.5, -0.10), loc='lower center', ncol=8, title='Models', labelspacing=0.4, handlelength=2, handletextpad=0.5)

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import seaborn as sns

# Set Seaborn style
sns.set_style('whitegrid')

metrics = ['U', 'K', 'PK', 'CK']
line_styles = ['-', '-', '--', ':']
colors = ['blue', 'black', 'orange', 'red']
block_width = 10  # Adjust the width of the color blocks

# Create subplots for U and another subplot for K, PK, CK
fig, axs = plt.subplots(1, 2, figsize=(8, 4), sharex=True)
fig.subplots_adjust(hspace=0.4)  # Adjust the vertical spacing between subplots

# Plot for U
ax_u = axs[0]
grouped_data_u = df.groupby(['Model', 'Shots'])['U'].agg(['mean', 'std']).reset_index()
handles_u = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    subset = grouped_data_u[grouped_data_u['Model'] == model]
    shots = subset['Shots']
    mean = subset['mean']
    std = subset['std']
    line = ax_u.plot(shots, mean, marker='o', linestyle=line_styles[0], color=colors[i], label=model)
    handles_u.append(line[0])

ax_u.set_ylabel('U-mAP')

# Plot for K, PK, CK
ax_kpkck = axs[1]
handles_kpkck = []
for j, metric in enumerate(['K', 'PK', 'CK']):
    grouped_data = df.groupby(['Model', 'Shots'])[metric].agg(['mean', 'std']).reset_index()
    for k, model in enumerate(grouped_data['Model'].unique()):
        subset = grouped_data[grouped_data['Model'] == model]
        shots = subset['Shots']
        mean = subset['mean']
        std = subset['std']
        line = ax_kpkck.plot(shots, mean, marker='o', linestyle=line_styles[j+1], color=colors[k])
        handles_kpkck.append(line[0])

ax_kpkck.set_ylabel('K-mAP')

# Create custom legend handles for line styles
line_handles = []
for style in line_styles:
    line = mlines.Line2D([], [], color='black', linestyle=style)
    line_handles.append(line)

# Create color block handles for models
model_handles = []
for i, model in enumerate(grouped_data_u['Model'].unique()):
    color = colors[i]
    patch = mpatches.Rectangle((0, 0), 1, 0.6, facecolor=color, linewidth=1, edgecolor='k')  # Adjust the width and height
    model_handles.append(patch)

# Combine the U, K, PK, CK legend handles
all_handles = model_handles + line_handles
labels = [model for model in grouped_data_u['Model'].unique()] + metrics

# Create legend
fig.legend(handles=all_handles, labels=labels, bbox_to_anchor=(0.53, -0.13), loc='lower center', ncol=8, title='Models', labelspacing=0.7, handlelength=1.7, handletextpad=0.3, columnspacing=0.9)
# Set the x-axis label for the last subplot
ax_kpkck.set_xlabel('Number of Shots')
ax_u.set_xlabel('Number of Shots')

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()
